Doing this tutorial: https://nbconvert.readthedocs.io/en/latest/nbconvert_library.html

In [1]:
from urllib.request import urlopen

url = 'https://jakevdp.github.io/downloads/notebooks/XKCD_plots.ipynb'
response = urlopen(url).read().decode()
response[0:60] + ' ...'

'{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "metadata": ...'

In [2]:
import nbformat
jake_notebook = nbformat.reads(response, as_version=4)
jake_notebook.cells[0]

{'cell_type': 'markdown',
 'metadata': {},
 'source': '# XKCD plots in Matplotlib'}

In [28]:
from traitlets.config import Config

# 1. Import the exporter
from nbconvert import HTMLExporter

# 2. Instantiate the exporter. We use the `classic` template for now; we'll get into more details
# later about how to customize the exporter further.
html_exporter = HTMLExporter(template_name='classic')

# 3. Process the notebook we loaded earlier
(body, resources) = html_exporter.from_notebook_node(jake_notebook)

In [18]:
print(body[:400] + '...')


<!DOCTYPE html>
<html>
<head><meta charset="utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1.0">

<title>Notebook</title><script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/2.0.3/jquery.min.js"></script><script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.1.10/require.min.js"></script>

<style type="text/css">
  pre { line-height: 125%; }
td.linenos ....


In [19]:
print("Resources:", resources.keys())
print("Metadata:", resources['metadata'].keys())
print("Inlining:", resources['inlining'].keys())
print("Extension:", resources['output_extension'])

Resources: dict_keys(['metadata', 'output_extension', 'deprecated', 'theme', 'include_css', 'include_js', 'include_url', 'require_js_url', 'mathjax_url', 'jquery_url', 'jupyter_widgets_base_url', 'widget_renderer_url', 'html_manager_semver_range', 'inlining', 'raw_mimetypes', 'global_content_filter'])
Metadata: dict_keys(['name'])
Inlining: dict_keys(['css'])
Extension: .html


### High Level Notebook Conversion Process

From [this](https://nbconvert.readthedocs.io/en/latest/nbconvert_library.html#Some-theory)

Before we get into actually extracting the figures, it will be helpful to give a high-level overview of the process of converting a notebook to a another format:

1. Retrieve the notebook and it’s accompanying resources (you are responsible for this).

2. Feed the notebook into the `Exporter`, which:
    
    a. Sequentially feeds the notebook into an array of `Preprocessors`. Preprocessors only act on the structure of the notebook, and have unrestricted access to it.
    
    b. Feeds the notebook into the Jinja templating engine, which converts it to a particular format depending on which template is selected.
    
5. The exporter returns the converted notebook and other relevant resources as a tuple.

6. You write the data to the disk using the built-in `FilesWriter` (which writes the notebook and any extracted files to disk), or elsewhere using a custom `Writer`.

### Creating Template files on the fly: means you don't have to store templates.

In [25]:
from jinja2 import DictLoader

dl = DictLoader({'footer2':
"""
{%- extends 'lab/index.html.j2' -%}

{% block footer %}
FOOOOOOOOTEEEEER
{% endblock footer %}
"""})


exportHTML = HTMLExporter(extra_loaders=[dl], template_file='footer2')
(body, resources) = exportHTML.from_notebook_node(jake_notebook)
for l in body.split('\n')[-4:]:
    print(l)


</body>

FOOOOOOOOTEEEEER



## Aside: WTF is [Traitlets](https://traitlets.readthedocs.io/en/stable/using_traitlets.html)

In [54]:
from traitlets import HasTraits, Int, Unicode, default, TraitError
import getpass

class Identity(HasTraits):
    username = Unicode()

    @default('username')
    def _default_username(self):
        return getpass.getuser() # getpass is in the standard lib and getuser() gets the user name.

Two things going on here:

1. the default value of `username` will be set to whatever is returned by the thing decorated by `@default('username')`.  The method can be named anything, but the convention is `_default_<property_name>`.
2. There will be input validation such that a `Trait` error will be raised if the input passed is of a different type.

In [55]:
i = Identity() # the default value is used
assert i.username == getpass.getuser()

In [64]:
try:
    i = Identity(username=3)
except TraitError as e:
    print(e)

The 'username' trait of an Identity instance expected a unicode string, not the int 3.


#### Fine, what about `traitlets.config.Config`